In [1]:
%run _BaseInfo.ipynb
import time

from openpyxl import load_workbook
def save_excel(file_name,df):
    sheet_name='Sheet1'
    try:
        # 嘗試加載現有的 Excel 文件
        book = load_workbook(file_name)
        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # 加載現有數據
            writer.book = book
            if sheet_name in book.sheetnames:
                existing_df = pd.read_excel(file_name, sheet_name=sheet_name)
                #df = pd.concat([existing_df, df], ignore_index=True)
            # 將 DataFrame 寫入指定的 Sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    except FileNotFoundError:
        # 如果文件不存在，創建新文件並寫入 DataFrame
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [2]:
df_data_all= pd.DataFrame()
stock_list = _baseInfo.dropna()['公司代號'].iloc[::-1]
for code in stock_list:
    print(code)
    try:
        time.sleep(1)
        url='https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a='+code
        print(url)
        response = requests.get(url,verify = False)
        response.encoding = 'big5'
        df = pd.DataFrame()
        df = pd.concat([df, pd.DataFrame([[str(code)]], columns=['stock_number'])], ignore_index=True)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            table=soup.find('table').find('table').find('table')

            for x in range(len(table.find_all('tr'))):
                #if x>=1 and x<5:
                if x!=0  and (x<12 or x==20) :
                    detail=[th.text.replace('', '') for th in table.find_all('tr')[x].find_all('td')]
                    for y in range(len(detail)-1):
                        if (y % 2) == 0:
                            df[detail[y]]=str(detail[y+1])

            save_df=df#[['stock_number', '收盤價' ,'每股淨值(元)','股價淨值比','本益比', '一年內最高價', '一年內最低價', '一年內最大量', '一年內最低量','營收比重']]
            df_data_all=pd.concat([ df_data_all,save_df], axis=0)

    except:
            print('error'+code)

9962
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=9962
9960
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=9960
9951
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=9951
9950
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=9950
9949
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=9949
8942
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8942
8941
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8941
8938
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8938
8937
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8937
8936
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8936
8935
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8935
8933
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8933
8932
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8932
8931
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8931
8930
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=8930
8929
https://pscnetinvest.moneydj.com/z/

https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6967
6953
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6953
6929
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6929
6922
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6922
6913
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6913
6904
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6904
6903
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6903
6899
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6899
6895
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6895
6894
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6894
6881
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6881
6877
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6877
6875
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6875
6874
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6874
6870
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6870
6865
https://pscnetinvest.moneydj.com/z/zc/zc

https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6425
6419
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6419
6418
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6418
6417
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6417
6411
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6411
6294
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6294
6292
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6292
6291
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6291
6290
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6290
6287
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6287
6284
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6284
6279
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6279
6276
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6276
6275
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6275
6274
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=6274
6270
https://pscnetinvest.moneydj.com/z/zc/zc

5514
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5514
5512
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5512
5511
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5511
5508
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5508
5498
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5498
5493
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5493
5490
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5490
5489
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5489
5488
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5488
5487
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5487
5483
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5483
5481
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5481
5478
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5478
5475
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5475
5474
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=5474
5468
https://pscnetinvest.moneydj.com/z/

https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4728
4726
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4726
4721
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4721
4716
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4716
4714
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4714
4711
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4711
4707
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4707
4706
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4706
4702
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4702
4609
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4609
4584
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4584
4580
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4580
4577
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4577
4568
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4568
4563
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=4563
4561
https://pscnetinvest.moneydj.com/z/zc/zc

...

3321
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3321
3312
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3312
3311
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3311
3308
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3308
3305
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3305
3296
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3296
3266
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3266
3257
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3257
3231
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3231
3229
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3229
3209
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3209
3189
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3189
3168
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3168
3167
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3167
3164
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=3164
3150
https://pscnetinvest.moneydj.com/z/

https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2722
2712
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2712
2707
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2707
2706
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2706
2705
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2705
2704
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2704
2702
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2702
2701
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2701
2646
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2646
2645
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2645
2642
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2642
2637
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2637
2636
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2636
2634
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2634
2633
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2633
2630
https://pscnetinvest.moneydj.com/z/zc/zc

2405
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2405
2404
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2404
2402
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2402
2401
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2401
2399
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2399
2397
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2397
2395
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2395
2393
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2393
2392
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2392
2390
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2390
2388
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2388
2387
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2387
2385
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2385
2383
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2383
2382
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=2382
2380
https://pscnetinvest.moneydj.com/z/

https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1905
1904
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1904
1903
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1903
1817
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1817
1810
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1810
1809
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1809
1808
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1808
1806
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1806
1805
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1805
1802
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1802
1795
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1795
1789
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1789
1786
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1786
1783
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1783
1776
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1776
1773
https://pscnetinvest.moneydj.com/z/zc/zc

1440
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1440
1439
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1439
1438
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1438
1437
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1437
1436
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1436
1435
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1435
1434
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1434
1432
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1432
1423
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1423
1419
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1419
1418
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1418
1417
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1417
1416
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1416
1414
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1414
1413
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=1413
1410
https://pscnetinvest.moneydj.com/z/

In [3]:
df_data_all

,stock_number,開盤價,最高價,最低價,收盤價,漲跌,一年內最高價,一年內最低價,本益比,一年內最大量,一年內最低量,成交量,同業平均本益比,一年來最高本益比,一年來最低本益比,盤後量,殖利率,總市值,85年來最高總市值,85年來最低總市值,投資報酬率(01/22),獲利能力(113.3Q),今年以來,每股淨值(元),營業毛利率,貝他值,最近一週,每人營收(仟元),營業利益率,標準差,最近一個月,每股營收(元),稅前淨利率,最近二個月,負債比例,資產報酬率,最近三個月,股價淨值比,股東權益報酬率,營收比重,獲利能力(113.1Q),獲利能力(113.4Q)
0,9962,14.05,14.1,14,14.1,-0.05,18.95,13.15,20.14,"2,561",13,61,22.77,22.36,10.12,0,7.8%,"1,272","3,457",363,財務比例(113.3Q),投資風險(01/22),-3.75%,12.18,8.06%,0.39,1.08%,N/A,4.14%,1.10%,-6.93%,6.42,3.68%,-9.32%,27.81%,1.13%,-10.19%,1.16,1.50%,不�袗�板78.84%、不�袗�料20.94%、客戶委託代工(OEM)0.22% (2023年),NaN,NaN
0,9960,25.25,25.25,25.25,25.25,-0.1,28.75,25.1,12.20,430,0,1,22.07,12.75,9.41,0,6.73%,848,"1,737",169,財務比例(113.3Q),投資風險(01/22),-0.98%,18.98,33.14%,0.19,0.00%,"1,857.00",9.10%,0.59%,-1.75%,5.81,12.46%,-2.88%,41.47%,1.63%,-5.25%,1.33,2.75%,高爾夫用品100.00% (2023年),NaN,NaN
0,9951,56,56.7,56,56.5,+0.4,73.9,52.7,12.75,766,6,59,41.41,14.29,11.40,0,6.02%,"4,232","16,478",346,財務比例(113.3Q),投資風險(01/22),5.61%,47.86,23.59%,0.43,1.99%,"1,173.00",5.51%,1.00%,5.02%,18.14,5.20%,-6.77%,37.52%,1.02%,-10.03%,1.18,1.60%,汽車捲簾99.58%、其他0.42% (2023年),NaN,NaN
0,9950,14.3,14.3,14.1,14.25,0,22.15,12.7,19.52,"3,915",20,28,14.46,23.84,2.18,0,0%,"2,390","4,830",443,財務比例(113.3Q),投資風險(01/22),-5.94%,8.86,27.34%,0.66,1.06%,747.00,5.40%,2.33%,-15.43%,4.35,2.21%,6.74%,66.39%,0.63%,3.26%,1.61,1.07%,行李箱92.29%、其他7.71% (2023年),NaN,NaN
0,9949,24.7,25,24.65,25,+0.3,27.7,16.35,N/A,785,1,28,52.03,N/A,N/A,1,0%,"1,101","2,574",397,財務比例(113.3Q),投資風險(01/22),-1.96%,8.09,64.31%,0.30,-1.77%,191.00,-36.40%,2.68%,2.04%,0.74,-33.19%,-3.47%,28.18%,-2.20%,0.81%,3.09,-2.92%,水晶玻璃藝品93.79%、其他5.51%、外購產品0.65%、配飾0.05% (2023年),NaN,NaN
0,8942,55.7,56.3,55.7,56.3,+0.2,75.8,50.8,13.22,"4,329",76,151,183.90,13.62,7.18,2,8.88%,"11,195","17,063",663,財務比例(113.3Q),投資風險(01/22),9.53%,48.66,32.25%,0.29,1.26%,N/A,9.19%,0.91%,4.84%,4.67,17.66%,0.92%,25.25%,0.84%,-1.47%,1.16,1.09%,複合板96.06%、其他3.94% (2023年),NaN,NaN
0,8941,67.1,67.8,65.1,67.8,+0.2,83.3,59,13.29,406,0,18,95.04,15.60,8.88,0,8.11%,"2,371","12,865",303,財務比例(113.3Q),投資風險(01/22),0.00%,26.36,28.63%,0.15,0.30%,N/A,6.68%,1.34%,1.50%,23.52,5.94%,1.35%,49.18%,2.48%,-1.02%,2.57,4.74%,瓦斯桶開關49.37%、戶外瓦斯烤肉爐48.33%、零組件及其他1.22%、高效能熱水器1....,NaN,NaN
0,8938,74.7,75,74.5,74.8,+0.1,87.3,69.1,14.75,"2,900",0,138,22.07,158.40,14.16,3,2.14%,"11,566","16,230","2,277",財務比例(113.3Q),投資風險(01/22),-0.40%,48.77,12.42%,0.46,2.47%,316.00,-0.97%,1.10%,3.89%,20.90,-4.67%,-0.80%,47.51%,-0.40%,-6.85%,1.53,-1.04%,GOLF球具67.04%、GOLF球19.66%、複合產品13.30% (2023年),NaN,NaN
0,8937,167,180,152.5,170,+1.5,180,20.05,531.25,"5,607",8,"3,001",183.90,548.44,28.31,21,0%,"15,694","16,202",555,財務比例(113.3Q),投資風險(01/22),14.48%,13.16,-4.10%,0.65,3.98%,490.00,-39.91%,3.65%,30.27%,0.50,-92.50%,13.33%,36.94%,-1.86%,107.06%,12.92,-3.41%,多功能休閒車58.13%、其他車種及零件24.10%、摩托車16.12%、其他投資1.65%...,NaN,NaN
0,8936,53.7,54.5,53.5,53.6,+0.1,99.6,45.75,17.07,"78,816",435,"1,499",183.90,32.01,15.35,25,2.8%,"13,297","22,873",303,財務比例(113.3Q),投資風險(01/22),11.20%,23.86,24.70%,1.34,7.31%,"3,492.00",20.30%,2.96%,7.63%,4.81,20.84%,1.71%,40.36%,1.63%,-15.63%,2.25,2.67%,工程承攬59.59%、延性鑄鐵管17.41%、其他8.66%、營運和維護6.56%、海淡水2...,NaN,NaN


In [4]:
df_data_all
file_name='Result/StockInfo.xlsx'
#save_excel(file_name,df_data_all)
df_data_all.to_excel(file_name, index=False)

In [5]:
df_data_all.drop_duplicates().to_excel(file_name, index=False)